In [27]:
from typing import Dict, Tuple
from dataclasses import field
from quam.core import QuamRoot, quam_dataclass
from quam.components.quantum_components import Qubit, QubitPair
from quam.components import IQChannel

@quam_dataclass
class Transmon(Qubit):
    xy: IQChannel = field(default=None)


@quam_dataclass
class QUAM(QuamRoot):
    qubits: Dict[str, Qubit] = field(default_factory=dict)
    qubit_pairs: Dict[Tuple[str, str], QubitPair] = field(default_factory=dict)

## Instantiate QUAM

In [29]:
machine = QUAM()

## Add qubits
q0 = machine.qubits["q0"] = Transmon(id="q0", xy=IQChannel(opx_output_I=("con1", 1, 1), opx_output_Q=("con1", 1, 2), frequency_converter_up=None))
q1 = machine.qubits["q1"] = Transmon(id="q1")

## Add qubit pairs
machine.qubit_pairs["q0@q1"] = QubitPair(qubit_control=q0.get_reference(), qubit_target=q1.get_reference())


Access qubit pair

In [12]:
q0 @ q1

QubitPair(id='q0@q1', macros={}, qubit_control=Transmon(id='q0', macros={}), qubit_target=Transmon(id='q1', macros={}))

Add pulse and pulse macro to qubit

In [20]:
from quam.components.pulses import SquarePulse
from quam.components.macro import PulseMacro

q0.xy.operations["x180"] = SquarePulse(amplitude=0.1, length=100)
q0.macros["x180"] = PulseMacro(pulse=q0.xy.operations["x180"].get_reference())

In [24]:
from qm.qua import program
from qm import generate_qua_script, qua

with qua.program() as prog:
    q0.apply("x180")

print(generate_qua_script(prog))


# Single QUA script generated at 2025-01-28 20:38:08.437463
# QUA library version: 1.2.1

from qm import CompilerOptionArguments
from qm.qua import *

with program() as prog:
    play("x180", "q0.xy")


config = None

loaded_config = None




## Creating operations

In [25]:
from quam.core.operation import OperationsRegistry

operations_registry = OperationsRegistry()

@operations_registry.register_operation
def x180(qubit: Qubit, **kwargs):
    qubit.xy.operations["x180"].play(**kwargs)

In [26]:
with qua.program() as prog:
    x180(q0)

print(generate_qua_script(prog))


# Single QUA script generated at 2025-01-28 20:40:16.420682
# QUA library version: 1.2.1

from qm import CompilerOptionArguments
from qm.qua import *

with program() as prog:
    play("x180", "q0.xy")


config = None

loaded_config = None




In [ ]:
from quam.core import quam_dataclass
from quam.core.macro import QuamMacro

@quam_dataclass
class CliffordMacro(QuamMacro):
    def apply(self, qubit: Qubit, clifford_idx: int, **kwargs):
        if clifford_idx == 0:
            qubit.apply("x180")
        elif clifford_idx == 1:
            qubit.apply("y180")
        elif clifford_idx == 2:
            qubit.apply("x90")
        elif clifford_idx == 3:
            qubit.apply("x270")
        ...